# 2: Spherical wind
-------------------

We use the model of WX Psc. proposed by [De Beck et al. (2010)](https://ui.adsabs.harvard.edu/abs/2010A%26A...523A..18D/abstract).

In [1]:
import numpy             as np
import scipy             as sp
import matplotlib.pyplot as plt
import magritte.tools    as tools
import magritte.setup    as setup
import magritte.core     as magritte

from astropy import units, constants

In [2]:
# Deifne names for the data files
model_file = 'output/magritte_spherical_wind.hdf5'
lamda_file = 'data/co.txt'

Download the line data file form the [LAMDA database](https://home.strw.leidenuniv.nl/~moldata/).

In [3]:
!wget "https://home.strw.leidenuniv.nl/~moldata/datafiles/co.dat" --output-document $lamda_file

--2024-02-15 15:52:16--  https://home.strw.leidenuniv.nl/~moldata/datafiles/co.dat
Resolving home.strw.leidenuniv.nl (home.strw.leidenuniv.nl)... 132.229.214.179
Connecting to home.strw.leidenuniv.nl (home.strw.leidenuniv.nl)|132.229.214.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444204 (434K)
Saving to: ‘data/co.txt’

data/co.txt         100%[===================>] 433.79K  --.-KB/s    in 0.1s    

2024-02-15 15:52:16 (2.89 MB/s) - ‘data/co.txt’ saved [444204/444204]



In [4]:
npoints  = 128

distance = (740     * units.pc            ).si.value
r_star   = (520     * constants.R_sun     ).si.value
r_in     =  1.7     * r_star
r_out    = (1.1e+17 * units.cm            ).si.value
v_inf    = (19.8    * units.km/units.s    ).si.value
T_star   = (2750    * units.K             ).si.value
epsilon  =  0.5
Mdot     = (1.9e-5  * units.M_sun/units.yr).si.value
xCO      =  1.0e-4
v_turb   = (1.5e+0  * units.km   /units.s ).si.value
T_star   = (2.5e+3  * units.K             ).si.value

rs = np.logspace(np.log10(r_in), np.log10(r_out), npoints, dtype=np.float64)

v_r = v_inf  * np.ones_like(rs)
trb = v_turb * np.ones_like(rs)
tmp = T_star * (r_star / rs)**epsilon
rho = Mdot / (4.0 * np.pi * rs**2 * v_r)
nH2 = (constants.N_A.si.value / 2.02e-3) * rho
nCO = xCO * nH2

In [5]:
model = magritte.Model ()
model.parameters.set_spherical_symmetry(True)
model.parameters.set_model_name        (model_file)
model.parameters.set_dimension         (1)
model.parameters.set_npoints           (npoints)
model.parameters.set_nrays             (12)
model.parameters.set_nspecs            (3)
model.parameters.set_nlspecs           (1)
model.parameters.set_nquads            (5)

model.geometry.points.position.set([[r,                      0, 0] for r in rs])
model.geometry.points.velocity.set([[v/constants.c.si.value, 0, 0] for v in v_r])

model.chemistry.species.abundance = [[CO, H2, 0.0] for (CO, H2) in zip(nCO, nH2)]
model.chemistry.species.symbol    = ['CO', 'H2', 'e-']

model.thermodynamics.temperature.gas  .set( tmp                         )
model.thermodynamics.turbulence.vturb2.set((trb/constants.c.si.value)**2)

model = setup.set_Delaunay_neighbor_lists (model)
model = setup.set_Delaunay_boundary       (model)
model = setup.set_boundary_condition_CMB  (model)
model = setup.set_rays_spherical_symmetry (model)
model = setup.set_linedata_from_LAMDA_file(model, lamda_file)
model = setup.set_quadrature              (model)

# Write and read model to initialize all variables
model.write()
model.read ()

Writing parameters...
Writing points...
Writing rays...
Writing boundary...
Writing chemistry...
Writing species...
Writing thermodynamics...
Writing temperature...
Writing turbulence...
Writing lines...
Writing lineProducingSpecies...
Writing linedata...
ncolpoar = 2
--- colpoar = 0
Writing collisionPartner...
(l, c) = 0, 0
--- colpoar = 1
Writing collisionPartner...
(l, c) = 0, 1
Writing quadrature...
Writing populations...
Writing radiation...
Writing frequencies...
                                           
-------------------------------------------
  Reading Model...                         
-------------------------------------------
 model file = output/magritte_spherical_wind.hdf5
-------------------------------------------
Reading parameters...
Reading points...
Reading rays...
Reading boundary...
Reading chemistry...
Reading species...
Reading thermodynamics...
Reading temperature...
Reading turbulence...
Reading lines...
Reading lineProducingSpecies...
Reading linedata...


In [6]:
# Initialize model with LTE level populations
model.compute_spectral_discretisation ()
model.compute_inverse_line_widths     ()
model.compute_LTE_level_populations   ()

Computing spectral discretisation...


0

Computing inverse line widths...
Computing LTE level populations...


In [ ]:
# Iterate level populations until statistical equilibrium
model.compute_level_populations_sparse (True, 200)

using ng acceleration? 0
Starting iteration 1
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 0.378408 seconds
Minor warning: population inversions detected; Magritte does not handle masers, so setting affected populations to LTE.
Tot | Compute Statistical Equilibrium : 0.003838 seconds
Already 19.6075 % converged!
using ng acceleration? 0
Starting iteration 2
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 0.523233 seconds
Minor warning: population inversions detected; Magritte does not handle masers, so setting affected populations to LTE.
Tot | Compute Statistical Equilibrium : 0.007176 seconds
Already 32.2599 % converged!
using ng acceleration? 0
Starting iteration 3
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
-

20

: 7 used order: 7
Minor warning: population inversions detected; Magritte does not handle masers, so setting affected populations to LTE.
using ng acceleration? 0
Starting iteration 17
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 2.860233 seconds
Minor warning: population inversions detected; Magritte does not handle masers, so setting affected populations to LTE.
Tot | Compute Statistical Equilibrium : 0.105558 seconds
Already 80.564 % converged!
using ng acceleration? 0
Starting iteration 18
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 3.007736 seconds
Minor warning: population inversions detected; Magritte does not handle masers, so setting affected populations to LTE.
Tot | Compute Statistical Equilibrium : 0.108788 seconds
Already 67.9306 % converged!
usi